In [3]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

pathname = 'D:\Dataset\data\WildScenes\WildScenes2d\V-01'
sample_size = 5

X_all = []
y_all = []

def one_hot_encode(masks, num_classes):
    masks_encoded = np.zeros((*masks.shape, num_classes), dtype=np.float32)
    for i in range(num_classes):
        masks_encoded[..., i] = (masks == i).astype(np.float32)
    return masks_encoded

for img_name in os.listdir(os.path.join(pathname, 'image'))[:sample_size]:
    if ':' in img_name:
        continue
    img_path = os.path.join(pathname, 'image', img_name)
    mask_path = os.path.join(pathname, 'indexLabel', img_name)
    
    img = np.asarray(cv2.imread(img_path)) / 255.0
    mask = np.asarray(cv2.imread(mask_path))
    mask = mask[:,:,0]
    
    X_all.append(img)
    y_all.append(mask)

X_all = np.asarray(X_all)
y_all = one_hot_encode(np.asarray(y_all), 15)
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Input, Cropping2D, ZeroPadding2D
from tensorflow.keras.models import Model

def unet_model(input_size=(1512, 2016, 3), num_classes=15):
    inputs = Input(input_size)

    # Encoding
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoding
    u5 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = concatenate([u5, c3])
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c2])
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c1])
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c7)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Create and compile the model
model = unet_model(input_size=(1512, 2016, 3), num_classes=15)

from tensorflow.keras.metrics import MeanIoU

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=[MeanIoU(num_classes=15)])

In [ ]:
print(y_train.shape)

model.fit(X_train, y_train, 
          validation_data=(X_val, y_val),
          batch_size=8, 
          epochs=20)

(3, 1512, 2016, 15)
Epoch 1/20


In [ ]:
def calculate_iou(pred_mask, true_mask, num_classes):
    pred_mask = pred_mask.flatten()
    true_mask = true_mask.flatten()
    
    # Create one-hot encoded masks
    pred_one_hot = (np.arange(num_classes) == pred_mask[:, None]).astype(int)
    true_one_hot = (np.arange(num_classes) == true_mask[:, None]).astype(int)
    
    # Calculate intersection and union
    intersection = np.logical_and(pred_one_hot, true_one_hot).sum(axis=0)
    union = np.logical_or(pred_one_hot, true_one_hot).sum(axis=0)
    
    # Calculate IoU for each class
    iou = np.divide(intersection, union, out=np.zeros_like(intersection, dtype=float), where=union != 0)
    
    mean_iou = np.mean(iou)
    return iou, mean_iou

def evaluate_segmentation(pred_masks, true_masks, num_classes):
    all_ious = np.array([calculate_iou(pred, true, num_classes)[0] for pred, true in zip(pred_masks, true_masks)])
    
    mean_ious = np.mean(all_ious, axis=0)
    overall_mean_iou = np.mean(mean_ious[mean_ious != 0])
    
    return mean_ious, overall_mean_iou

y_pred_one_hot = np.argmax(y_pred, axis=-1)
y_test_one_hot = np.argmax(y_test, axis=-1)

num_classes = 15
class_ious, mean_iou = evaluate_segmentation(y_pred_one_hot, y_test_one_hot, num_classes)

print("IoU for each class:", class_ious)
print("Mean IoU:", mean_iou)